# Plastic Optimization

We attempt to design new plastic molecule using LDM.
Our main goal is on
1. Straw
2. ?

## Setup enviroment
!git clone https://github.com/Ahnd6474/KSEF

%cd your_shit/GitHub/KSEF

!pip install -r requirements.txt

In [19]:
import pandas as pd
from __future__ import annotations

import os
import pickle
from pathlib import Path
from typing import Iterable, Iterator, Optional
import pyarrow
import torch
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset, load_model, sampling, visualize_molecule_3d

## Read Data

In [20]:
df=pd.read_parquet('data/plastic.parquet')
df

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2,smiles,num_side,num_back,end_group,names
0,0.0,246.341049,354.051544,524.699646,773.208252,25.838213,32.409279,103.836082,-0.074397,-0.109570,0.493981,-0.071000,-0.061771,0.296051,"[[*]OCCC(=O)[*], [*]OCCC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, ]","[pha, pha]"
1,0.1,313.798401,449.257751,502.616241,1866.776855,44.379429,38.010651,7.215131,-0.043089,-0.006026,0.462577,-0.049233,-0.031425,0.342873,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
2,0.2,306.681305,446.618713,502.605896,1769.267456,44.743877,37.998493,8.727348,-0.050900,-0.028856,0.455602,-0.060321,-0.039934,0.322175,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
3,0.3,298.431488,442.813019,503.664642,1653.973633,44.287182,37.676380,10.997544,-0.057812,-0.048113,0.458959,-0.069126,-0.047438,0.314363,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
4,0.4,290.073944,436.398560,505.013275,1508.705688,42.840149,36.907192,14.239414,-0.065853,-0.065725,0.461121,-0.075789,-0.055182,0.302125,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373498,0.0,325.252380,462.438263,502.465759,2680.520020,39.839355,68.497833,30.515642,0.011635,-0.060768,0.179605,0.002471,0.001415,0.056135,"[[*]CC([*])O, [*]CC([*])O]","[nan, nan]","[nan, nan]","[None, None]","[PVA, PVA]"
1373499,0.0,369.788818,509.966248,587.491455,2227.901367,33.554047,37.865055,1.950041,2.547638,11.337365,21.572718,0.559539,0.881040,24.865118,"[[*]CC([*])c1ccccc1, [*]CC([*])c1ccccc1]","[nan, nan]","[nan, nan]","[None, None]","[PS, PS]"
1373500,0.0,344.375183,457.250885,574.055664,1724.992798,50.262169,46.697941,27.839941,0.099392,0.332474,1.581858,-0.005775,0.070204,1.482780,"[[*]CC([*])Cl, [*]CC([*])Cl]","[nan, nan]","[nan, nan]","[None, None]","[PVC, PVC]"
1373501,0.0,393.653473,536.671326,700.403137,1942.565186,44.805931,65.647507,27.729816,0.016741,0.138303,1.767082,-0.054595,-0.069961,1.336078,"[[*]CCOC(=O)c1ccc2cc(C(=O)O[*])ccc2c1, [*]CCOC...","[nan, nan]","[nan, nan]","[None, None]","[PEN, PEN]"


In [21]:
df.describe()

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2
count,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06
mean,4.997987e-01,2.725839e+02,3.529060e+02,5.769558e+02,5.405507e+02,2.073843e+01,2.460826e+01,2.115185e+02,8.110294e-01,4.332410e+00,7.026700e+00,1.901237e-01,4.661335e-01,7.116223e+00
std,2.583418e-01,2.701165e+01,3.713276e+01,2.686434e+01,4.882776e+02,1.366027e+01,1.229784e+01,1.205617e+02,8.184573e-01,3.972706e+00,4.018037e+00,2.560846e-01,5.097858e-01,4.612499e+00
min,0.000000e+00,2.020853e+02,2.794441e+02,4.246918e+02,9.102757e+01,3.014123e+00,4.558969e+00,4.952888e-01,-9.569907e-02,-1.267213e-01,1.728067e-01,-1.294968e-01,-1.326250e-01,2.026558e-05
25%,3.000000e-01,2.543430e+02,3.266345e+02,5.673138e+02,1.744876e+02,8.976903e+00,1.493889e+01,1.076303e+02,3.791428e-01,1.926220e+00,4.530061e+00,5.799794e-02,1.869081e-01,4.429616e+00
50%,5.000000e-01,2.699055e+02,3.456665e+02,5.827677e+02,3.266322e+02,1.742608e+01,2.180278e+01,2.111784e+02,6.681085e-01,3.541990e+00,6.793644e+00,1.420151e-01,3.605243e-01,6.750260e+00
75%,7.000000e-01,2.883478e+02,3.725729e+02,5.933018e+02,7.870456e+02,3.001149e+01,3.317859e+01,3.043738e+02,1.066986e+00,5.741556e+00,9.030967e+00,2.563907e-01,6.126437e-01,9.138975e+00
max,9.000000e-01,4.033092e+02,5.529254e+02,7.004031e+02,3.081163e+03,7.795893e+01,1.298841e+02,5.666409e+02,3.425747e+01,1.253688e+02,1.188544e+02,8.895615e+00,1.651693e+01,1.671498e+02


## Property Prediction
We use MLP multi regressor to predict property of plastic

### Load Model and helpers

In [76]:

from __future__ import annotations

import ast
import pickle
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

from geoldm import encode, decode, load_model, smiles_to_3d, structure_to_smiles, visualize_molecule_3d
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset

TARGET_COLUMNS = [
    "Tg",
    "Tm",
    "Td",
    "YM",
    "TS_y",
    "TS_b",
    "eps_b",
    "perm_O2",
    "perm_CO2",
    "perm_He",
    "perm_N2",
    "perm_CH4",
    "perm_H2",
]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def load_qm9_latent_diffusion(checkpoint_dir: Path):
    """Load a pretrained latent diffusion model and its dataset metadata."""

    args_path = checkpoint_dir / "args.pickle"
    with args_path.open("rb") as handle:
        args = pickle.load(handle)

    # Respect GPU availability explicitly.
    setattr(args, "cuda", torch.cuda.is_available())

    dataset_info = get_dataset_info(args.dataset, args.remove_h)
    dataloaders, _ = dataset.retrieve_dataloaders(args)
    train_loader = dataloaders["train"]

    model, nodes_dist, _ = load_model(
        stage="latent_diffusion",
        args=args,
        dataset_info=dataset_info,
        dataloader_train=train_loader,
        checkpoint_path=checkpoint_dir,
    )
    model = model.to(DEVICE)
    model.eval()
    device = next(model.parameters()).device
    return model, dataset_info, nodes_dist, device


def _clean_smiles_string(smiles: str) -> str:
    """Replace placeholder atoms with hydrogens and trim whitespace."""

    return smiles.replace("[*]", "").strip()


def _pick_base_smiles(smiles_entry: object) -> str:
    """Normalize the parquet smiles field to a single SMILES string."""

    def iterate_candidates(seq):
        for candidate in seq:
            if isinstance(candidate, str):
                cleaned = _clean_smiles_string(candidate)
                if cleaned:
                    return cleaned
        return None

    if isinstance(smiles_entry, str):
        try:
            maybe_list = ast.literal_eval(smiles_entry)
        except Exception:
            maybe_list = None

        if isinstance(maybe_list, (list, tuple, np.ndarray)):
            candidate = iterate_candidates(maybe_list)
            if candidate:
                return candidate

        bracketed = smiles_entry.strip()
        if bracketed.startswith("[") and bracketed.endswith("]"):
            inner = bracketed[1:-1]
            parts = [part.strip() for part in inner.split(",") if part.strip()]
            candidate = iterate_candidates(parts)
            if candidate:
                return candidate

        return _clean_smiles_string(smiles_entry)

    if isinstance(smiles_entry, (list, tuple, np.ndarray)):
        candidate = iterate_candidates(smiles_entry)
        if candidate:
            return candidate

    return str(smiles_entry)


def conformer_to_tensors(
    conformer: Dict,
    dataset_info: Dict,
    device: torch.device,
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor], torch.Tensor, torch.Tensor]:
    """Convert a single RDKit conformer into GeoLDM-ready tensors."""

    atom_decoder: List[str] = dataset_info["atom_decoder"]
    atom_encoder = {symbol: idx for idx, symbol in enumerate(atom_decoder)}
    atom_indices = torch.tensor(
        [atom_encoder[symbol] for symbol in conformer["atom_symbols"]],
        dtype=torch.long,
        device=device,
    )

    one_hot = F.one_hot(atom_indices, num_classes=len(atom_decoder)).float()
    positions = torch.tensor(conformer["coordinates"], dtype=torch.float32, device=device)

    # Add batch dimensions and masks.
    x = positions.unsqueeze(0)
    h = {
        "categorical": one_hot.unsqueeze(0),
        # Use zero charges when absent.
        "integer": torch.zeros(one_hot.shape[0], 1, device=device).unsqueeze(0),
    }

    node_mask = torch.ones(x.shape[0], x.shape[1], 1, device=device)
    edge_mask = node_mask.squeeze(-1)[..., None] * node_mask.squeeze(-1)[:, None]
    return x, h, node_mask, edge_mask


def _pick_monomer_pair(smiles_entry: object) -> tuple[str, str]:
    """Extract two monomer SMILES strings from a heterogeneous entry.

    The plastics dataset stores SMILES in a mix of formats (lists, bracketed
    strings, raw strings). This helper mirrors ``_pick_base_smiles`` but returns
    the first two cleaned SMILES tokens when available. If only one token exists
    it is duplicated so callers can still form a valid pair.
    """

    def iterate_candidates(seq):
        cleaned = []
        for candidate in seq:
            if isinstance(candidate, str):
                val = _clean_smiles_string(candidate)
                if val:
                    cleaned.append(val)
            if len(cleaned) >= 2:
                break
        return cleaned

    if isinstance(smiles_entry, str):
        try:
            maybe_list = ast.literal_eval(smiles_entry)
        except Exception:
            maybe_list = None

        if isinstance(maybe_list, (list, tuple, np.ndarray)):
            cleaned = iterate_candidates(maybe_list)
            if cleaned:
                return (cleaned + [cleaned[-1]])[:2]

        bracketed = smiles_entry.strip()
        if bracketed.startswith("[") and bracketed.endswith("]"):
            inner = bracketed[1:-1]
            parts = [part.strip() for part in inner.split(",") if part.strip()]
            cleaned = iterate_candidates(parts)
            if cleaned:
                return (cleaned + [cleaned[-1]])[:2]

        cleaned = _clean_smiles_string(smiles_entry)
        return (cleaned, cleaned)

    if isinstance(smiles_entry, (list, tuple, np.ndarray)):
        cleaned = iterate_candidates(smiles_entry)
        if cleaned:
            return (cleaned + [cleaned[-1]])[:2]

    fallback = _clean_smiles_string(str(smiles_entry))
    return (fallback, fallback)


def _flatten_and_pad_latent(z: torch.Tensor, *, max_nodes: Optional[int] = None) -> torch.Tensor:
    """Flatten a per-node latent tensor, padding the node dimension when needed."""

    if max_nodes is not None and z.shape[1] < max_nodes:
        pad = max_nodes - z.shape[1]
        z = F.pad(z, (0, 0, 0, pad, 0, 0))
    return z.flatten(start_dim=1).cpu()


@torch.no_grad()
def smiles_to_latent(
    smiles: str,
    model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    *,
    return_structure: bool = False,
    max_nodes: Optional[int] = None,
) -> tuple[torch.Tensor, torch.Tensor] | tuple[tuple[torch.Tensor, torch.Tensor], Dict[str, Any]]:
    """Encode a SMILES string into node-level latent tensors (no pooling).

    The latents are flattened per-node embeddings so downstream code can feed
    ``z_x`` and ``z_h`` separately to the surrogate without averaging.
    """

    model.eval()
    conformers = smiles_to_3d(smiles)
    if len(conformers) == 0:
        raise ValueError(f"Failed to build 3D structure for {smiles!r}")

    conformer = conformers[0]
    x, h, node_mask, edge_mask = conformer_to_tensors(conformer, dataset_info, device)
    z_x, _, z_h, _ = encode(model, x, h, node_mask=node_mask, edge_mask=edge_mask)

    latent_x = _flatten_and_pad_latent(z_x, max_nodes=max_nodes)
    latent_h = _flatten_and_pad_latent(z_h, max_nodes=max_nodes)

    if return_structure:
        structure = {
            "smiles": smiles,
            "atom_symbols": conformer["atom_symbols"],
            "coordinates": np.asarray(conformer["coordinates"], dtype=np.float32),
        }
        return (latent_x, latent_h), structure

    return latent_x, latent_h


def build_latent_matrix(
    frame: pd.DataFrame,
    model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    *,
    sample_size: int = 512,
    batch_size: int = 32,
    cache_path: Optional[Path] = Path("data/plastic_latents.pt"),
    alpha_column: str = "comp",
) -> Tuple[torch.Tensor, torch.Tensor, List[int], List[Dict[str, Any]], List[Tuple[str, str]], torch.Tensor]:
    """Convert a dataframe of plastics into latent vectors, targets, and cached structures.

    The helper optionally caches results to avoid recomputation when the notebook
    is rerun. Each entry stores the 3D conformer (atom symbols + coordinates)
    associated with the embedding so we can retrieve structures directly without
    performing a nearest-SMILES lookup. When a legacy cache predates structure
    storage, the helper reconstructs the SMILES list so downstream utilities can
    regenerate structures on demand without rerunning the full encoding pass.
    """

    if cache_path and cache_path.exists():
        payload = torch.load(cache_path)
        cached_pairs = payload.get("smiles_pairs")
        if cached_pairs is None:
            base_smiles_pairs = [
                _pick_monomer_pair(frame.iloc[idx]["smiles"]) for idx in payload["indices"]
            ]
        else:
            base_smiles_pairs = cached_pairs

        alpha_values = payload.get("alpha", torch.tensor([0.5] * len(base_smiles_pairs)))

        return (
            payload["embeddings"],
            payload["targets"],
            payload["indices"],
            payload.get("structures", []),
            base_smiles_pairs,
            alpha_values,
        )

    subset = frame.sample(min(sample_size, len(frame)), random_state=0).reset_index()
    embeddings: List[torch.Tensor] = []
    targets: List[torch.Tensor] = []
    used_indices: List[int] = []
    structures: List[Dict[str, Any]] = []
    base_smiles: List[Tuple[str, str]] = []
    alpha_values: List[float] = []

    model.eval()
    progress = tqdm(total=len(subset), desc="Encoding SMILES", unit="sample")
    for start in range(0, len(subset), batch_size):
        batch = subset.iloc[start : start + batch_size]
        batch_payloads: List[Tuple[Tuple[str, str], float, np.ndarray, int]] = []

        for _, row in batch.iterrows():
            monomer_pair = _pick_monomer_pair(row["smiles"])
            alpha_value = float(row.get(alpha_column, 0.5))
            target_values = (
                pd.to_numeric(row[TARGET_COLUMNS], errors="coerce").astype(np.float32).to_numpy()
            )
            if not np.isfinite(target_values).all():
                continue
            batch_payloads.append((monomer_pair, alpha_value, target_values, int(row["index"])))

        batch_latents: List[torch.Tensor] = []
        batch_targets: List[torch.Tensor] = []
        batch_indices: List[int] = []
        batch_structures: List[Dict[str, Any]] = []
        batch_smiles: List[Tuple[str, str]] = []
        batch_alpha: List[float] = []

        max_nodes = dataset_info.get("max_n_nodes")

        for (smiles_one, smiles_two), alpha_value, target_values, row_index in batch_payloads:
            try:
                (latent_x1, latent_h1), structure1 = smiles_to_latent(
                    smiles_one,
                    model,
                    dataset_info,
                    device,
                    return_structure=True,
                    max_nodes=max_nodes,
                )
                latent_x2, latent_h2 = smiles_to_latent(
                    smiles_two,
                    model,
                    dataset_info,
                    device,
                    max_nodes=max_nodes,
                )
            except Exception:
                continue
            alpha_tensor = torch.tensor([[alpha_value]], dtype=latent_x1.dtype, device=latent_x1.device)
            latent_mix = torch.cat(
                [latent_x1, latent_h1, latent_x2, latent_h2, alpha_tensor], dim=-1
            ).squeeze(0)
            batch_latents.append(latent_mix)
            batch_targets.append(torch.tensor(target_values, dtype=torch.float32))
            batch_indices.append(row_index)
            batch_structures.append({"monomer1": structure1, "monomer2": smiles_two})
            batch_smiles.append((smiles_one, smiles_two))
            batch_alpha.append(alpha_value)

        embeddings.extend(batch_latents)
        targets.extend(batch_targets)
        used_indices.extend(batch_indices)
        structures.extend(batch_structures)
        base_smiles.extend(batch_smiles)
        alpha_values.extend(batch_alpha)
        progress.update(len(batch))

    progress.close()

    if not embeddings:
        raise RuntimeError("No embeddings were created; check SMILES parsing")

    emb_tensor = torch.stack(embeddings)
    target_tensor = torch.stack(targets)

    if cache_path:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(
            {
                "embeddings": emb_tensor,
                "targets": target_tensor,
                "indices": used_indices,
                "structures": structures,
                "smiles_pairs": base_smiles,
                "alpha": torch.tensor(alpha_values, dtype=torch.float32),
            },
            cache_path,
        )

    return (
        emb_tensor,
        target_tensor,
        used_indices,
        structures,
        base_smiles,
        torch.tensor(alpha_values, dtype=torch.float32),
    )


def find_nearest_structures(
    latent: torch.Tensor,
    embeddings: torch.Tensor,
    structures: List[Dict[str, Any]],
    *,
    top_k: int = 1,
    base_smiles: Optional[List[object]] = None,
) -> List[Dict[str, Any]]:
    """Locate the nearest cached structures to a latent vector via Euclidean distance."""

    if structures and embeddings.shape[0] != len(structures):
        raise ValueError("embeddings and structures must have matching lengths")

    if not structures:
        if base_smiles is None:
            raise ValueError("base_smiles are required when structures are not cached")
        if len(base_smiles) != embeddings.shape[0]:
            raise ValueError("base_smiles must align with embeddings")

        def build_structure(idx: int) -> Dict[str, Any]:
            smiles_entry = base_smiles[idx]
            if isinstance(smiles_entry, tuple):
                smiles_value = smiles_entry[0]
            else:
                smiles_value = smiles_entry
            conformers = smiles_to_3d(smiles_value)
            if not conformers:
                raise RuntimeError(f"Failed to regenerate structure for {smiles_value}")
            conformer = conformers[0]
            return {
                "smiles": smiles_value,
                "atom_symbols": conformer["atom_symbols"],
                "coordinates": np.asarray(conformer["coordinates"], dtype=np.float32),
            }
    else:
        def build_structure(idx: int) -> Dict[str, Any]:
            return dict(structures[idx])

    latent = latent.detach().cpu()
    embeddings_cpu = embeddings.detach().cpu()

    dists = torch.cdist(latent.unsqueeze(0), embeddings_cpu)[0]
    topk = torch.topk(dists, k=min(top_k, len(dists)), largest=False)

    results = []
    for idx, dist in zip(topk.indices.tolist(), topk.values.tolist()):
        entry = build_structure(idx)
        entry["distance"] = float(dist)
        results.append(entry)
    return results


def save_structure_xyz(atom_symbols: List[str], coordinates: np.ndarray, path: Path, *, title: str = "") -> None:
    """Persist a simple XYZ file for downstream visualization or sharing."""

    path.parent.mkdir(parents=True, exist_ok=True)
    header = f"{len(atom_symbols)}{title}"
    lines = [f"{sym} {x:.6f} {y:.6f} {z:.6f}" for sym, (x, y, z) in zip(atom_symbols, coordinates)]
    with path.open("w") as fp:
        fp.write(header)
        fp.write("".join(lines))
        fp.write("")


def predict_from_latent(
    latent: torch.Tensor,
    *,
    mlp_model: nn.Module,
    x_stats: Dict[str, torch.Tensor],
    y_stats: Dict[str, torch.Tensor],
) -> Dict[str, float]:
    """Run the surrogate MLP on a latent vector and return property predictions."""

    props = surrogate_forward(latent, mlp_model=mlp_model, x_stats=x_stats, y_stats=y_stats)
    return {name: tensor.detach().cpu().item() for name, tensor in props.items()}


class MultiTaskMLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        hidden_sizes=(512, 512, 256),
        dropout: float = 0.0,
    ):
        super().__init__()
        self.hidden_sizes = tuple(hidden_sizes)
        self.dropout = dropout
        layers = []
        prev = input_dim
        for width in hidden_sizes:
            layers.append(nn.Linear(prev, width))
            layers.append(nn.SiLU())  # or GELU
            if dropout > 0.0:
                layers.append(nn.Dropout(dropout))
            prev = width
        layers.append(nn.Linear(prev, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


def save_mlp_checkpoint(
    checkpoint_path: Path,
    model: MultiTaskMLP,
    x_scaler: StandardScaler,
    y_scaler: StandardScaler,
    *,
    epoch: int,
    val_loss: float,
) -> None:
    """Persist the MLP weights and scalers for reuse."""

    checkpoint_path = Path(checkpoint_path)
    checkpoint_path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(
        {
            "epoch": epoch,
            "val_loss": val_loss,
            "input_dim": model.network[0].in_features,
            "output_dim": model.network[-1].out_features,
            "hidden_sizes": model.hidden_sizes,
            "dropout": model.dropout,
            "state_dict": model.state_dict(),
            "x_scaler": x_scaler,
            "y_scaler": y_scaler,
        },
        checkpoint_path,
    )


def load_mlp_checkpoint(
    checkpoint_path: Path, device: Optional[torch.device] = None
) -> Tuple[MultiTaskMLP, StandardScaler, StandardScaler, Dict]:
    """Load a saved multitask MLP and its associated scalers.

    PyTorch 2.6 defaults ``weights_only`` to ``True``, which blocks arbitrary
    objects such as the ``StandardScaler`` instances stored in our checkpoint.
    Explicitly disable that restriction when we trust the checkpoint source.
    """

    # Allow the saved sklearn scalers to be restored when loading trusted files.
    torch.serialization.add_safe_globals([StandardScaler])

    checkpoint = torch.load(
        checkpoint_path,
        map_location=device or DEVICE,
        weights_only=False,
    )
    model = MultiTaskMLP(
        checkpoint["input_dim"],
        checkpoint["output_dim"],
        hidden_sizes=checkpoint.get("hidden_sizes", (512, 512, 256)),
        dropout=checkpoint.get("dropout", 0.0),
    )
    model.load_state_dict(checkpoint["state_dict"])
    model.to(device or DEVICE)
    return model, checkpoint["x_scaler"], checkpoint["y_scaler"], checkpoint



def train_multitask_mlp(
    embeddings: torch.Tensor,
    targets: torch.Tensor,
    *,
    epochs: int = 15,
    batch_size: int = 64,
    lr: float = 1e-3,
    validation_split: float = 0.1,
    log_dir: Optional[str] = "runs/plastic_mlp",
    checkpoint_path: Optional[Path] = Path("models/plastic_mlp_best.pt"),
    device: Optional[torch.device] = None,
) -> Tuple[MultiTaskMLP, StandardScaler, StandardScaler, Dict[str, List[float]]]:
    """Train a simple multitask MLP on latent embeddings and log to TensorBoard."""

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    x_scaled = x_scaler.fit_transform(embeddings)
    y_scaled = y_scaler.fit_transform(targets)

    features = torch.tensor(x_scaled, dtype=torch.float32)
    labels = torch.tensor(y_scaled, dtype=torch.float32)

    dataset = TensorDataset(features, labels)
    val_size = max(1, int(len(dataset) * validation_split))
    train_size = len(dataset) - val_size
    train_set, val_set = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)

    model = MultiTaskMLP(features.shape[1], labels.shape[1]).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    writer = SummaryWriter(log_dir=log_dir) if log_dir else None
    history = {"train": [], "val": []}
    best_val = float("inf")
    best_state: Optional[Dict[str, torch.Tensor]] = None
    for epoch in tqdm(range(epochs), desc="Training epochs"):
        model.train()
        train_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(batch_x)
        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                preds = model(batch_x)
                loss = criterion(preds, batch_y)
                val_loss += loss.item() * len(batch_x)
        val_loss /= len(val_loader.dataset)

        history["train"].append(train_loss)
        history["val"].append(val_loss)
        if writer:
            writer.add_scalar("loss/train", train_loss, epoch)
            writer.add_scalar("loss/val", val_loss, epoch)

        if val_loss < best_val:
            best_val = val_loss
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            if checkpoint_path:
                save_mlp_checkpoint(
                    checkpoint_path,
                    model,
                    x_scaler,
                    y_scaler,
                    epoch=epoch + 1,
                    val_loss=val_loss,
                )
            print(f"Saved new best model at epoch {epoch + 1}: val_loss={val_loss:.6f}")
            if writer:
                writer.add_scalar("loss/best_val", best_val, epoch)

    if writer:
        writer.flush()
        writer.close()

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, x_scaler, y_scaler, history


def predict_properties(
    monomer1_smiles: str,
    monomer2_smiles: Optional[str],
    alpha: float,
    *,
    ldm_model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    mlp_model: MultiTaskMLP,
    x_scaler: StandardScaler,
    y_scaler: StandardScaler,
) -> Dict[str, float]:
    """Generate property predictions for a monomer pair + composition."""

    max_nodes = dataset_info.get("max_n_nodes")
    latent_x1, latent_h1 = smiles_to_latent(
        monomer1_smiles, ldm_model, dataset_info, device, max_nodes=max_nodes
    )
    latent_x2, latent_h2 = smiles_to_latent(
        monomer2_smiles or monomer1_smiles,
        ldm_model,
        dataset_info,
        device,
        max_nodes=max_nodes,
    )
    alpha_tensor = torch.tensor([[alpha]], dtype=latent_x1.dtype, device=latent_x1.device)
    latent = torch.cat([latent_x1, latent_h1, latent_x2, latent_h2, alpha_tensor], dim=-1).squeeze(0)
    scaled = x_scaler.transform(latent.unsqueeze(0))
    mlp_model = mlp_model.to(device)
    mlp_model.eval()
    with torch.no_grad():
        pred_scaled = mlp_model(torch.tensor(scaled, dtype=torch.float32, device=device)).cpu().numpy()
    pred = y_scaler.inverse_transform(pred_scaled)[0]
    return {name: value for name, value in zip(TARGET_COLUMNS, pred)}



In [77]:
# Load pretrained latent diffusion encoder
checkpoint_dir = Path("./qm9_latent2")
ldm_model, dataset_info, nodes_dist, device = load_qm9_latent_diffusion(checkpoint_dir)
print(f"Using device: {device}")

# Prepare the plastics dataframe and build (or load) latent embeddings.
plastic_df = pd.read_parquet("data/plastic.parquet")
embeddings, targets, used_indices, structures, base_smiles_pairs, alpha_values = build_latent_matrix(
    plastic_df,
    ldm_model,
    dataset_info,
    device,
    sample_size=1373503,  # 1373503
    cache_path=Path("data/plastic_latents.pt"),
)

# Train the multi-task MLP regressor and persist the best checkpoint.
checkpoint_path = Path("models/plastic_mlp_best.pt")
mlp_model, x_scaler, y_scaler, history = train_multitask_mlp(
    embeddings.numpy(),
    targets.numpy(),
    epochs=300,
    batch_size=64,
    lr=1e-4,
    checkpoint_path=checkpoint_path,
    device=device,
)

print("Training loss trajectory:", history["train"])
print("Validation loss trajectory:", history["val"])

# Reload the best-performing weights and scalers from disk for downstream use.
mlp_model, x_scaler, y_scaler, checkpoint_meta = load_mlp_checkpoint(
    checkpoint_path, device=device
)
print(
    f"Loaded checkpoint from epoch {checkpoint_meta['epoch']} "
    f"with val_loss={checkpoint_meta['val_loss']:.6f}"
)


Entropy of n_nodes: H[N] -2.475700616836548
Autoencoder models are _not_ conditioned on time.
alphas2 [9.99990000e-01 9.99988000e-01 9.99982000e-01 ... 2.59676966e-05
 1.39959211e-05 1.00039959e-05]
gamma [-11.51291546 -11.33059532 -10.92513058 ...  10.55863126  11.17673063
  11.51251595]
Using device: cuda:0


Encoding SMILES:   0%|          | 0/1373503 [00:00<?, ?sample/s]

[21:30:25] SMILES Parse Error: syntax error while parsing: CC()Cl
[21:30:25] SMILES Parse Error: check for mistakes around position 4:
[21:30:25] CC()Cl
[21:30:25] ~~~^
[21:30:25] SMILES Parse Error: Failed parsing SMILES 'CC()Cl' for input: 'CC()Cl'
[21:30:30] SMILES Parse Error: syntax error while parsing: CC()c1ccccc1
[21:30:30] SMILES Parse Error: check for mistakes around position 4:
[21:30:30] CC()c1ccccc1
[21:30:30] ~~~^
[21:30:30] SMILES Parse Error: Failed parsing SMILES 'CC()c1ccccc1' for input: 'CC()c1ccccc1'
[21:30:35] SMILES Parse Error: syntax error while parsing: CC()CCCC
[21:30:35] SMILES Parse Error: check for mistakes around position 4:
[21:30:35] CC()CCCC
[21:30:35] ~~~^
[21:30:35] SMILES Parse Error: Failed parsing SMILES 'CC()CCCC' for input: 'CC()CCCC'
[21:30:47] SMILES Parse Error: syntax error while parsing: CC()CCCC
[21:30:47] SMILES Parse Error: check for mistakes around position 4:
[21:30:47] CC()CCCC
[21:30:47] ~~~^
[21:30:47] SMILES Parse Error: Failed pars

RuntimeError: stack expects each tensor to be equal size, but got [311] at entry 0 and [371] at entry 1

In [44]:
# Reload the best-performing weights and scalers from disk for downstream use.
mlp_model, x_scaler, y_scaler, checkpoint_meta = load_mlp_checkpoint(
    checkpoint_path, device=device
)
print(
    f"Loaded checkpoint from epoch {checkpoint_meta['epoch']} "
    f"with val_loss={checkpoint_meta['val_loss']:.6f}"
)


Loaded checkpoint from epoch 92 with val_loss=0.636485


[예측값 정의]

logP_CO2  = log(P_CO2)
logP_CH4  = log(P_CH4)
logP_N2   = log(P_N2)

Tg        = 유리전이온도
Td        = 열분해온도
YM        = Young's modulus
TSb       = 인장강도 (stress at break)
epsb      = 연신율 (strain at break)

z1, z2    = 두 모노머의 latent vector


[1. 분리 성능 보상항 J_sep]

J_sep =
    w_perm      * logP_CO2
  + w_sel_CH4   * (logP_CO2 - logP_CH4)   # CO2/CH4 선택성 (log S_CO2/CH4)
  + w_sel_N2    * (logP_CO2 - logP_N2)    # CO2/N2  선택성 (log S_CO2/N2)


[2. 고온·고압용 물성 soft penalty]

임계값(조건)을 다음처럼 둔다 (예시는 너가 quantile 보고 채우면 됨):

Tg_req     = 원하는 최소 Tg
Td_req     = 원하는 최소 Td
YM_min     = 원하는 최소 YM
TSb_min    = 원하는 최소 TSb
epsb_min   = 원하는 최소 epsb

각 penalty는 "부족한 만큼만 제곱"으로 정의:

pen_Tg  = max(0, Tg_req  - Tg)^2
pen_Td  = max(0, Td_req  - Td)^2
pen_YM  = max(0, YM_min  - YM)^2
pen_TS  = max(0, TSb_min - TSb)^2
pen_eps = max(0, epsb_min - epsb)^2


[3. latent prior penalty]

pen_z = ||z1||^2 + ||z2||^2
# (local search면 ||z1 - z1_0||^2 + ||z2 - z2_0||^2 로 바꿔도 됨)


[4. 전체 목적함수 J]

J =
    J_sep
  - λ_Tg  * pen_Tg
  - λ_Td  * pen_Td
  - λ_YM  * pen_YM
  - λ_TS  * pen_TS
  - λ_eps * pen_eps
  - λ_z   * pen_z


[5. 최종 최적화에 쓰는 loss]

loss = -J


### Adam surrogate optimization (Adam)

`Property Prediction` 섹션에서 학습된 MLP를 서로게이트 모델로 사용해, 위 마크다운에 정의된 목표식을 Adam으로 최적화합니다.

In [58]:

# Adam-based latent optimization using the trained MLP surrogate

# Constraint/weight defaults follow the markdown objective; thresholds use the upper quartile
# of the observed properties so that the optimizer is nudged toward the stronger end.
quantiles = plastic_df[TARGET_COLUMNS].quantile(0.75)
default_constraint_config = {
    "Tg_req": float(quantiles.get("Tg", 0.0)),
    "Td_req": float(quantiles.get("Td", 0.0)),
    "YM_min": float(quantiles.get("YM", 0.0)),
    "TSb_min": float(quantiles.get("TS_b", 0.0)),
    "epsb_min": float(quantiles.get("eps_b", 0.0)),
}

# λ coefficients for each soft constraint and the latent prior
default_lambda_config = {"Tg": 1.0, "Td": 1.0, "YM": 1.0, "TS": 1.0, "eps": 1.0, "z": 1e-2}

# Weights for the separation reward
default_weight_config = {"perm": 1.0, "sel_CH4": 1.0, "sel_N2": 1.0}


def _scaler_to_tensors(scaler, device):
    return {
        "mean": torch.tensor(scaler.mean_, device=device, dtype=torch.float32),
        "scale": torch.tensor(scaler.scale_, device=device, dtype=torch.float32),
    }


def surrogate_forward(latent, *, mlp_model, x_stats, y_stats):
    """Run the frozen surrogate model on a latent vector and return a property dict (torch tensors)."""

    latent_scaled = (latent - x_stats["mean"]) / x_stats["scale"]
    preds_scaled = mlp_model(latent_scaled)
    preds = preds_scaled * y_stats["scale"] + y_stats["mean"]
    return {name: preds[:, idx] for idx, name in enumerate(TARGET_COLUMNS)}


def surrogate_loss(
    z1,
    z2,
    *,
    mlp_model,
    x_stats,
    y_stats,
    weight_config,
    constraint_config,
    lambda_config,
):
    device = z1.device
    latent = torch.cat([z1, z2], dim=1)
    props = surrogate_forward(latent, mlp_model=mlp_model, x_stats=x_stats, y_stats=y_stats)

    logP_CO2 = torch.log(torch.clamp(props["perm_CO2"], min=1e-9))
    logP_CH4 = torch.log(torch.clamp(props["perm_CH4"], min=1e-9))
    logP_N2 = torch.log(torch.clamp(props["perm_N2"], min=1e-9))

    J_sep = (
        weight_config["perm"] * logP_CO2
        + weight_config["sel_CH4"] * (logP_CO2 - logP_CH4)
        + weight_config["sel_N2"] * (logP_CO2 - logP_N2)
    )

    Tg_req = torch.tensor(constraint_config["Tg_req"], device=device)
    Td_req = torch.tensor(constraint_config["Td_req"], device=device)
    YM_min = torch.tensor(constraint_config["YM_min"], device=device)
    TSb_min = torch.tensor(constraint_config["TSb_min"], device=device)
    epsb_min = torch.tensor(constraint_config["epsb_min"], device=device)

    pen_Tg = F.relu(Tg_req - props["Tg"]) ** 2
    pen_Td = F.relu(Td_req - props["Td"]) ** 2
    pen_YM = F.relu(YM_min - props["YM"]) ** 2
    pen_TS = F.relu(TSb_min - props["TS_b"]) ** 2
    pen_eps = F.relu(epsb_min - props["eps_b"]) ** 2

    pen_z = (z1 ** 2).sum(dim=1) + (z2 ** 2).sum(dim=1)

    J = (
        J_sep
        - lambda_config["Tg"] * pen_Tg
        - lambda_config["Td"] * pen_Td
        - lambda_config["YM"] * pen_YM
        - lambda_config["TS"] * pen_TS
        - lambda_config["eps"] * pen_eps
        - lambda_config["z"] * pen_z
    )

    loss = -J.mean()
    metrics = {
        "loss": loss.detach(),
        "J": J.mean().detach(),
        "J_sep": J_sep.mean().detach(),
        "pen_Tg": pen_Tg.mean().detach(),
        "pen_Td": pen_Td.mean().detach(),
        "pen_YM": pen_YM.mean().detach(),
        "pen_TS": pen_TS.mean().detach(),
        "pen_eps": pen_eps.mean().detach(),
        "pen_z": pen_z.mean().detach(),
    }
    return loss, metrics


def optimize_latent_with_adam(
    *,
    mlp_model,
    x_stats,
    y_stats,
    weight_config=default_weight_config,
    constraint_config=default_constraint_config,
    lambda_config=default_lambda_config,
    steps: int = 300,
    lr: float = 5e-3,
    seed: int = 42,
):
    torch.manual_seed(seed)
    device = next(mlp_model.parameters()).device
    mlp_model.eval()

    latent_dim = mlp_model.network[0].in_features
    split = latent_dim // 2
    z1 = torch.randn(1, split, device=device, requires_grad=True)
    z2 = torch.randn(1, latent_dim - split, device=device, requires_grad=True)

    optimizer = torch.optim.Adam([z1, z2], lr=lr)
    history = []
    best = {"loss": float("inf"), "state": None, "metrics": None, "step": -1}

    for step in range(1, steps + 1):
        optimizer.zero_grad()
        loss, metrics = surrogate_loss(
            z1,
            z2,
            mlp_model=mlp_model,
            x_stats=x_stats,
            y_stats=y_stats,
            weight_config=weight_config,
            constraint_config=constraint_config,
            lambda_config=lambda_config,
        )
        loss.backward()
        optimizer.step()

        metrics = {k: v.detach().item() for k, v in metrics.items()}
        metrics["step"] = step
        history.append(metrics)

        if loss.item() < best["loss"]:
            best = {
                "loss": loss.item(),
                "state": (z1.detach().clone(), z2.detach().clone()),
                "metrics": metrics,
                "step": step,
            }

    best_z1, best_z2 = best["state"]
    best_latent = torch.cat([best_z1, best_z2], dim=1)
    return best_latent, best["metrics"], history


# Prepare scaler tensors for differentiable preprocessing/inverse-transform
x_stats = _scaler_to_tensors(x_scaler, device)
y_stats = _scaler_to_tensors(y_scaler, device)

best_latent, best_metrics, opt_history = optimize_latent_with_adam(
    mlp_model=mlp_model,
    x_stats=x_stats,
    y_stats=y_stats,
    steps=200,
    lr=5e-3,
)

print(f"Best objective J={best_metrics['J']:.4f} at step {best_metrics['step']}")
print({k: v for k, v in best_metrics.items() if k not in {"step"}})


Best objective J=-193954079838104977408.0000 at step 199
{'loss': 1.9395407983810498e+20, 'J': -1.9395407983810498e+20, 'J_sep': 20.450267791748047, 'pen_Tg': 7.484864417549517e+16, 'pen_Td': 1.0345689047619338e+18, 'pen_YM': 1.9283229650279675e+20, 'pen_TS': 1.2370764237897728e+16, 'pen_eps': 0.0, 'pen_z': 8.421366691589355}


In [66]:
print(best_latent)

tensor([[ 0.1940,  2.1614,  0.1391,  0.8868, -1.7124]], device='cuda:0')


In [60]:
candidate_latents = {"optimized": best_latent.detach().cpu()}
noise = torch.randn(4, best_latent.shape[1])
scales = [0.05, -0.05, 0.1, -0.1]
for idx, scale in enumerate(scales, start=1):
    candidate_latents[f"neighbor_{idx}"] = best_latent.detach().cpu() + scale * noise[idx - 1]
candidate_latents

{'optimized': tensor([[ 0.1940,  2.1614,  0.1391,  0.8868, -1.7124]]),
 'neighbor_1': tensor([[ 0.2904,  2.2357,  0.1842,  0.7815, -1.7503]]),
 'neighbor_2': tensor([[ 0.1401,  2.1213,  0.0551,  0.8690, -1.6781]]),
 'neighbor_3': tensor([[ 0.1447,  2.1855,  0.1160,  0.8910, -1.7376]]),
 'neighbor_4': tensor([[ 0.1080,  2.1923,  0.1787,  0.8064, -1.6503]])}

In [65]:
from geoldm import (
    run_diffusion,
    smiles_to_3d,
    structure_to_smiles,
    visualize_molecule_3d,
)

# 예시 SMILES
smiles = "CCO"

# 1) SMILES -> 3D conformer 목록
conformers = smiles_to_3d(smiles)
if len(conformers) == 0:
    raise ValueError(f"Failed to build 3D structure for {smiles!r}")

# 첫 번째 conformer 사용
conformer = conformers[0]

# 2) GeoLDM용 텐서로 변환
x, h, node_mask, edge_mask = conformer_to_tensors(conformer, dataset_info, device)

# 3) 노드 개수는 x에서 직접 가져오는 게 제일 안전함
n_nodes = x.size(1)          # = node_mask.shape[1]
n_samples = 1

# 4) latent diffusion에서 샘플링
sampled_positions, decoded_features = run_diffusion(
    ldm_model,
    n_samples=n_samples,
    n_nodes=n_nodes,
    node_mask=node_mask,
    edge_mask=edge_mask,
    context=None,
    fix_noise=True,
)

# 5) 3D 구조 → SMILES
atom_types = decoded_features["categorical"].argmax(dim=-1)[0].cpu().numpy()
atom_symbols = [dataset_info["atom_decoder"][i] for i in atom_types]
new_smiles, _ = structure_to_smiles(atom_symbols, sampled_positions[0].cpu().numpy())
print(f"Generated SMILES: {new_smiles}")


ValueError: Final molecular charge (0) does not match input (9); could not find valid bond ordering

## 노드-space 두 모노머 최적화
MLP 서러게이트 입력이 이미 `concat([latent_m1, latent_m2, alpha])` 형태로 학습되었다고 가정하고,\n모노머 각각의 노드 latent(`z_x, z_h`)와 조성비 α(logit 형태)를 직접 최적화하는 루프를 추가한다.\nGeoLDM encoder에서 나온 노드 latent를 `pool_node_latent`로 풀링한 뒤 MLP 입력으로 사용한다.


**주의 (훈련 세팅)**: 위에서 추가한 노드-space + 조성비 α 최적화 루프는
MLP가 `concat([latent_m1, latent_m2, alpha])` 입력으로 미리 학습되어 있다는
가정 위에서만 동작한다. 현재 노트북의 학습 파이프라인은 `build_latent_matrix`가
단일 모노머 latent를 풀링(mean)해 만든 벡터를 바로 `train_multitask_mlp`에 넣어
스케일링·학습하기 때문에 입력 차원이 `(d_x + d_h)` 하나짜리로 고정돼 있다.
따라서 α까지 포함한 두 모노머 입력을 쓰려면 **새 입력 차원**으로 데이터/모델을
다시 준비해 학습해야 한다.


## 노드-space 두 모노머 최적화
MLP 서러게이트 입력이 이미 `concat([latent_m1, latent_m2, alpha])` 형태로 학습되었다고 가정하고,\n모노머 각각의 노드 latent(`z_x, z_h`)와 조성비 α(logit 형태)를 직접 최적화하는 루프를 추가한다.\nGeoLDM encoder에서 나온 노드 latent를 `pool_node_latent`로 풀링한 뒤 MLP 입력으로 사용한다.


In [ ]:
def smiles_to_full_latent(
    smiles: str,
    ldm_model,
    dataset_info: Dict,
    device: torch.device,
):
    """
    단일 모노머 SMILES -> (z_x, z_h, node_mask, edge_mask)

    z_x: [1, N, d_x]
    z_h: [1, N, d_h]
    node_mask: [1, N, 1]
    edge_mask: [1, N, N]
    """
    conformers = smiles_to_3d(smiles)
    if len(conformers) == 0:
        raise ValueError(f"Failed to build 3D structure for {smiles!r}")

    conformer = conformers[0]
    x, h, node_mask, edge_mask = conformer_to_tensors(conformer, dataset_info, device)

    with torch.no_grad():
        z_x, _, z_h, _ = encode(
            ldm_model,
            x,
            h,
            node_mask=node_mask,
            edge_mask=edge_mask,
        )

    return z_x, z_h, node_mask, edge_mask


def pool_node_latent(
    z_x: torch.Tensor,
    z_h: torch.Tensor,
    node_mask: torch.Tensor,
) -> torch.Tensor:
    """
    z_x: [B, N, d_x]
    z_h: [B, N, d_h]
    node_mask: [B, N, 1]

    반환: [B, d_x + d_h]
    (plastic.ipynb에서 single SMILES latent를 만들 때 사용한 pooling과 동일하게 맞춰야 함)
    """
    mask = node_mask  # [B, N, 1]

    z_x_masked = z_x * mask
    z_h_masked = z_h * mask

    n_nodes = mask.sum(dim=1)  # [B, 1]

    pooled_x = z_x_masked.sum(dim=1) / n_nodes
    pooled_h = z_h_masked.sum(dim=1) / n_nodes

    pooled = torch.cat([pooled_x, pooled_h], dim=-1)  # [B, d_x + d_h]
    return pooled


def surrogate_loss_two_monomer_node_latent(
    z_x1: torch.Tensor,
    z_h1: torch.Tensor,
    node_mask1: torch.Tensor,
    z_x2: torch.Tensor,
    z_h2: torch.Tensor,
    node_mask2: torch.Tensor,
    mix_logit: torch.Tensor,
    *,
    mlp_model: torch.nn.Module,
    x_stats: Dict[str, torch.Tensor],
    y_stats: Dict[str, torch.Tensor],
    weight_config: Dict[str, float],
    constraint_config: Dict[str, float],
    lambda_config: Dict[str, float],
):
    """
    두 모노머의 노드 latent + 조성비를 직접 최적화할 때 쓰는 surrogate loss.

    - monomer 1: (z_x1, z_h1, node_mask1)
    - monomer 2: (z_x2, z_h2, node_mask2)
    - alpha = sigmoid(mix_logit)
    - MLP 입력: concat([latent_m1, latent_m2, alpha])
    ""

    device = z_x1.device

    # 1) 노드 latent -> monomer별 pooled latent
    latent1 = pool_node_latent(z_x1, z_h1, node_mask1)  # [1, D_m]
    latent2 = pool_node_latent(z_x2, z_h2, node_mask2)  # [1, D_m]

    # 2) 조성비: 0~1 범위로 제한
    alpha = torch.sigmoid(mix_logit)  # [1, 1]

    # 3) 최종 MLP 입력
    latent_mix = torch.cat([latent1, latent2, alpha], dim=1)  # [1, 2*D_m + 1]

    # 4) surrogate MLP를 통해 물성 예측
    props = surrogate_forward(latent_mix, mlp_model=mlp_model, x_stats=x_stats, y_stats=y_stats)

    # 분리 성능 항 (로그 투과도 + 선택도)
    logP_CO2 = torch.log(torch.clamp(props["perm_CO2"], min=1e-9))
    logP_CH4 = torch.log(torch.clamp(props["perm_CH4"], min=1e-9))
    logP_N2 = torch.log(torch.clamp(props["perm_N2"], min=1e-9))

    J_sep = (
        weight_config["perm"] * logP_CO2
        + weight_config["sel_CH4"] * (logP_CO2 - logP_CH4)
        + weight_config["sel_N2"] * (logP_CO2 - logP_N2)
    )

    # 제약 soft penalty
    Tg_req = torch.tensor(constraint_config["Tg_req"], device=device)
    Td_req = torch.tensor(constraint_config["Td_req"], device=device)
    YM_min = torch.tensor(constraint_config["YM_min"], device=device)
    TSb_min = torch.tensor(constraint_config["TS_b"], device=device)
    epsb_min = torch.tensor(constraint_config["eps_b"], device=device)

    pen_Tg = F.relu(Tg_req - props["Tg"]) ** 2
    pen_Td = F.relu(Td_req - props["Td"]) ** 2
    pen_YM = F.relu(YM_min - props["YM"]) ** 2
    pen_TS = F.relu(TSb_min - props["TS_b"]) ** 2
    pen_eps = F.relu(epsb_min - props["eps_b"]) ** 2

    pen_z1 = (latent1 ** 2).mean(dim=1)
    pen_z2 = (latent2 ** 2).mean(dim=1)
    pen_alpha = (mix_logit ** 2).mean(dim=1)

    pen_z = pen_z1 + pen_z2 + pen_alpha

    J = (
        J_sep
        - lambda_config["Tg"] * pen_Tg
        - lambda_config["Td"] * pen_Td
        - lambda_config["YM"] * pen_YM
        - lambda_config["TS"] * pen_TS
        - lambda_config["eps"] * pen_eps
        - lambda_config["z"] * pen_z
    )

    loss = -J.mean()

    metrics = {
        "loss": loss.detach(),
        "J": J.mean().detach(),
        "J_sep": J_sep.mean().detach(),
        "pen_Tg": pen_Tg.mean().detach(),
        "pen_Td": pen_Td.mean().detach(),
        "pen_YM": pen_YM.mean().detach(),
        "pen_TS": pen_TS.mean().detach(),
        "pen_eps": pen_eps.mean().detach(),
        "pen_z": pen_z.mean().detach(),
        "alpha": alpha.mean().detach(),
    }
    return loss, metrics


def optimize_two_monomer_in_node_space(
    *,
    seed_smiles1: str,
    seed_smiles2: str,
    ldm_model: torch.nn.Module,
    dataset_info: Dict,
    device: torch.device,
    mlp_model: torch.nn.Module,
    x_stats: Dict[str, torch.Tensor],
    y_stats: Dict[str, torch.Tensor],
    weight_config: Dict[str, float],
    constraint_config: Dict[str, float],
    lambda_config: Dict[str, float],
    steps: int = 300,
    lr: float = 5e-3,
    seed: int = 42,
):
    """
    모노머1/2의 노드 latent + 조성비 α 를 Adam으로 최적화.

    - 초기값: seed_smiles1, seed_smiles2 를 encode한 z_x, z_h
    - 변수: z_x1, z_h1, z_x2, z_h2, mix_logit
    - 최종: 최적 latent에서 각 모노머 구조 decode + α 반환
    ""

    torch.manual_seed(seed)
    ldm_model.eval()
    mlp_model.eval()

    z_x1_init, z_h1_init, node_mask1, edge_mask1 = smiles_to_full_latent(
        seed_smiles1, ldm_model, dataset_info, device
    )
    z_x2_init, z_h2_init, node_mask2, edge_mask2 = smiles_to_full_latent(
        seed_smiles2, ldm_model, dataset_info, device
    )

    z_x1_opt = z_x1_init.clone().detach().requires_grad_(True)
    z_h1_opt = z_h1_init.clone().detach().requires_grad_(True)
    z_x2_opt = z_x2_init.clone().detach().requires_grad_(True)
    z_h2_opt = z_h2_init.clone().detach().requires_grad_(True)

    mix_logit = torch.zeros(1, 1, device=device, requires_grad=True)  # 초기 α ≈ 0.5

    optimizer = torch.optim.Adam(
        [z_x1_opt, z_h1_opt, z_x2_opt, z_h2_opt, mix_logit],
        lr=lr,
    )

    best = {"J": -float("inf"), "state": None, "metrics": None, "step": -1}
    history: List[Dict[str, float]] = []

    for step in range(steps):
        optimizer.zero_grad()

        loss, metrics = surrogate_loss_two_monomer_node_latent(
            z_x1_opt,
            z_h1_opt,
            node_mask1,
            z_x2_opt,
            z_h2_opt,
            node_mask2,
            mix_logit,
            mlp_model=mlp_model,
            x_stats=x_stats,
            y_stats=y_stats,
            weight_config=weight_config,
            constraint_config=constraint_config,
            lambda_config=lambda_config,
        )

        loss.backward()
        optimizer.step()

        J_val = metrics["J"].item()
        alpha_val = metrics["alpha"].item()

        hist_row = {k: float(v.cpu()) for k, v in metrics.items()}
        hist_row["step"] = step
        history.append(hist_row)

        if J_val > best["J"]:
            best["J"] = J_val
            best["state"] = {
                "z_x1": z_x1_opt.detach().clone(),
                "z_h1": z_h1_opt.detach().clone(),
                "z_x2": z_x2_opt.detach().clone(),
                "z_h2": z_h2_opt.detach().clone(),
                "node_mask1": node_mask1.detach().clone(),
                "edge_mask1": edge_mask1.detach().clone(),
                "node_mask2": node_mask2.detach().clone(),
                "edge_mask2": edge_mask2.detach().clone(),
                "mix_logit": mix_logit.detach().clone(),
            }
            best["metrics"] = hist_row
            best["step"] = step

        if (step + 1) % 50 == 0 or step == 0:
            print(
                f"[{step+1:4d}/{steps}] "
                f"loss={metrics['loss']:.4f} "
                f"J={metrics['J']:.4f} J_sep={metrics['J_sep']:.4f} "
                f"alpha={alpha_val:.3f} "
                f"pen_Tg={metrics['pen_Tg']:.4f} pen_Td={metrics['pen_Td']:.4f} "
                f"pen_YM={metrics['pen_YM']:.4f} pen_TS={metrics['pen_TS']:.4f} "
                f"pen_eps={metrics['pen_eps']:.4f} pen_z={metrics['pen_z']:.4f}"
            )

    if best["state"] is None:
        raise RuntimeError("Optimization failed (no best state).")

    print(f"Best J={best['J']:.4f} at step={best['step']}")

    state = best["state"]
    mix_logit_best = state["mix_logit"]
    alpha_best = torch.sigmoid(mix_logit_best).item()

    with torch.no_grad():
        pos1, feat1 = decode(
            ldm_model,
            (state["z_x1"], state["z_h1"]),
            node_mask=state["node_mask1"],
            edge_mask=state["edge_mask1"],
        )
        pos2, feat2 = decode(
            ldm_model,
            (state["z_x2"], state["z_h2"]),
            node_mask=state["node_mask2"],
            edge_mask=state["edge_mask2"],
        )

    atom_types1 = feat1["categorical"].argmax(dim=-1)[0].cpu().numpy()
    atom_symbols1 = [dataset_info["atom_decoder"][i] for i in atom_types1]
    smiles1, _ = structure_to_smiles(atom_symbols1, pos1[0].cpu().numpy())

    atom_types2 = feat2["categorical"].argmax(dim=-1)[0].cpu().numpy()
    atom_symbols2 = [dataset_info["atom_decoder"][i] for i in atom_types2]
    smiles2, _ = structure_to_smiles(atom_symbols2, pos2[0].cpu().numpy())

    print(f"Optimized monomer 1 SMILES: {smiles1}")
    print(f"Optimized monomer 2 SMILES: {smiles2}")
    print(f"Optimized composition alpha (monomer1 fraction): {alpha_best:.3f}")

    visualize_molecule_3d(
        atom_symbols1,
        pos1[0].cpu().numpy(),
        out_html="optimized_monomer1.html",
        show=False,
    )
    visualize_molecule_3d(
        atom_symbols2,
        pos2[0].cpu().numpy(),
        out_html="optimized_monomer2.html",
        show=False,
    )

    return {
        "best_J": best["J"],
        "best_step": best["step"],
        "best_metrics": best["metrics"],
        "alpha": alpha_best,
        "z_x1": state["z_x1"],
        "z_h1": state["z_h1"],
        "z_x2": state["z_x2"],
        "z_h2": state["z_h2"],
        "node_mask1": state["node_mask1"],
        "edge_mask1": state["edge_mask1"],
        "node_mask2": state["node_mask2"],
        "edge_mask2": state["edge_mask2"],
        "pos1": pos1,
        "feat1": feat1,
        "pos2": pos2,
        "feat2": feat2,
        "smiles1": smiles1,
        "smiles2": smiles2,
        "history": history,
    }


In [ ]:
# 두 모노머 latent + 조성비 α를 직접 최적화
x_stats = _scaler_to_tensors(x_scaler, device)
y_stats = _scaler_to_tensors(y_scaler, device)

two_monomer_result = optimize_two_monomer_in_node_space(
    seed_smiles1="CCO",
    seed_smiles2="CCN",
    ldm_model=ldm_model,
    dataset_info=dataset_info,
    device=device,
    mlp_model=mlp_model,
    x_stats=x_stats,
    y_stats=y_stats,
    weight_config=default_weight_config,
    constraint_config=default_constraint_config,
    lambda_config=default_lambda_config,
    steps=300,
    lr=5e-3,
)

two_monomer_result
